In [69]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd

spark = sphynx.get_spark(executor_count=8, app_name='pdu_syp', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: pdu_syp...
Waiting for all executors ready...
3/8 executors ready
All executors connected!
Complete! elapsed time: 00:00:37


In [103]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [26]:
import boto3
client = boto3.client('s3')
bucket_path = "s3a://pubg-log-labs/seoyoung"

In [ ]:
client.upload_file(file_name, bucket)

pre_load_data_mart: 2021-07-28 이전 (나만의 상점 업데이트 전)  
load_data_mart: 2021-07-28 이후

In [31]:
# user 지역
from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())

meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

# df_user: user_master
# df_user = df_user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
# df_user = df_user.join(meta_region, df_user.country_new == meta_region.country_code_iso2, "left")

In [2]:
DATA_PATH = './data/'

In [6]:
all_economy_gcoin = mysql.read_table(spark, "pubg_gi", "all_economy_gcoin_used_by_product_daily")

In [99]:
item_meta_sales = mysql.read_table(spark, "metainfo", "item_meta_sales")

### EMOTE MAR2

#### 매출 Trend
- 지역별 
- 플랫폼 별

"""
select *
from pubg_gi.all_economy_gcoin_used_by_product_daily a
where product_id in (select product_id from metainfo.item_meta_sales where event_name = "2021_emote_mar2")
"""

In [63]:
mar2_daily = all_economy_gcoin.where(col("product_id").isin(
    item_meta_sales.where(col("event_name") == "2021_emote_mar2").select("product_id").distinct().toPandas()["product_id"].tolist()
)).toPandas()

In [111]:
mar2_daily.to_csv(DATA_PATH + 'mar2_daily.csv', index=False)
# mar2_daily = pd.read_csv(DATA_PATH + 'mar2_daily.csv')

In [48]:
mar2_period_gcoin_pc = pre_load_data_mart(start_date="2021-03-13", end_date="2021-06-12", device='pc', table_name='gcoin_use')
mar2_period_gcoin_console = pre_load_data_mart(start_date="2021-03-18", end_date="2021-06-17", device='console', table_name='gcoin_use')

In [49]:
pc_start_date = "2021-03-13"
pc_end_date = "2021-06-12"

console_start_date = "2021-03-18"
console_end_date = "2021-06-17"

mar2_user_pc = pre_load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='user_master').where(col("lastlogindate") >= pc_start_date)
mar2_gcoin_pc = pre_load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use').where(col("event_name") == '2021_emote_mar2')
print('== PC DONE ==')

mar2_user_console = pre_load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='user_master').where(col("lastlogindate") >= console_start_date)
mar2_gcoin_console = pre_load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use').where(col("event_name") == '2021_emote_mar2')
print('== CONSOLE DONE ==')

== PC DONE ==
== CONSOLE DONE ==


In [57]:
mar2_pc_a = mar2_user_pc.groupBy('platform').agg(countDistinct('accountid').alias('au')).toPandas()

In [61]:
mar2_pc_b = mar2_gcoin_pc.groupBy("platform").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()

In [63]:
mar2_pc = pd.merge(mar2_pc_a, mar2_pc_b, on='platform', how="left").fillna(0)
mar2_pc['pur'] = mar2_pc.pu / mar2_pc.au
mar2_pc['arpu'] = mar2_pc.revenue / mar2_pc.au
mar2_pc['arppu'] = mar2_pc.revenue / mar2_pc.pu

In [64]:
mar2_pc

,platform,au,cnt,pu,revenue,pur,arpu,arppu
0,KAKAO,1230657,12551.0,12551.0,7530600.0,0.010199,6.119170,600.0
1,STEAM,10743438,108748.0,108748.0,65248800.0,0.010122,6.073363,600.0
2,MAILRU,10668,14.0,14.0,8400.0,0.001312,0.787402,600.0
3,DMM,3558,0.0,0.0,0.0,0.000000,0.000000,NaN


In [75]:
mar2_pc.to_csv(DATA_PATH+ "mar2_pc.csv", index=False)

In [73]:
mar2_console_a = mar2_user_console.groupBy('platform').agg(countDistinct('accountid').alias('au')).toPandas()
mar2_console_b = mar2_gcoin_console.groupBy("platform").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()
mar2_console = pd.merge(mar2_console_a, mar2_console_b, on='platform', how="left").fillna(0)
mar2_console['pur'] = mar2_console.pu / mar2_console.au
mar2_console['arpu'] = mar2_console.revenue / mar2_console.au
mar2_console['arppu'] = mar2_console.revenue / mar2_console.pu

In [76]:
mar2_console.to_csv(DATA_PATH +"mar2_console.csv", index=False)

In [77]:
mar2_console

,platform,au,cnt,pu,revenue,pur,arpu,arppu
0,XBOX,1377224,8183,8183,4909800,0.005942,3.564997,600.0
1,STADIA,45517,47,47,28200,0.001033,0.619549,600.0
2,PSN,785802,3417,3417,2050200,0.004348,2.609054,600.0


In [161]:
mar2_user_pc.groupBy('country_os', "country_ip").agg(countDistinct('accountid').alias('au')).show()

+----------+----------+----+
|country_os|country_ip|  au|
+----------+----------+----+
|        ES|        NI|  83|
|        US|        NG|  46|
|        SK| Undefined|   3|
|        SG|        SG| 177|
|        SA|        RU|  11|
|        UA|        RU|  29|
|        SG|        JP|  11|
|        RU|        SI|  10|
|        SA|        CN|   2|
| Undefined|        PR|  15|
|        JO|        PS|   8|
|        CZ|        FI|   2|
|        PL|        HU|   1|
|        TW|        HU|   1|
|        PH|        CA|   1|
|        ES|        UY|1690|
|        GB|        BE| 460|
|        RU|        CZ| 284|
|        MX|        SV| 136|
|        ZA|        KR|  28|
+----------+----------+----+
only showing top 20 rows



### 해당 기간동안 다른 상품들과 비교

#### 해당 기간동안 다른 emote 상품들과 비교

In [111]:
mar2_period_gcoin_pc.where(col("sub_category") == "emotes").select(countDistinct("product_id")).show()

+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                        15|
+--------------------------+



* 날짜별 emote상품별로 판매량 나타내주면 좋을듯

In [81]:
mar2_period_gcoin_pc.where(col("sub_category") == "emotes").groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas()

,product_name,product_id,price,sell_cnt,PU,revenue,ARPPU
0,Victory Dance - LEENALCHI x JINJO CREW,itemdesc.16100250,600.0,121313,121313,72787800,600.000000
1,Victory Dance 41,itemdesc.16100141,500.0,60761,60761,30380500,500.000000
2,Victory Dance 49,itemdesc.16100259,500.0,56780,56780,28390000,500.000000
3,Victory Dance 47,itemdesc.16100257,500.0,55100,55100,27550000,500.000000
4,Victory Dance 44,itemdesc.16100254,400.0,47544,47543,19017600,400.008413
5,Victory Dance 46,itemdesc.16100256,500.0,45417,45417,22708500,500.000000
6,Victory Dance 48,itemdesc.16100258,500.0,37911,37911,18955500,500.000000
7,Victory Dance 45,itemdesc.16100255,500.0,33567,33567,16783500,500.000000
8,Victory Dance 50,itemdesc.16100260,500.0,31928,31928,15964000,500.000000
9,Victory Dance 39,itemdesc.16100129,400.0,25032,25031,10012800,400.015980


In [82]:
mar2_period_gcoin_console.where(col("sub_category") == "emotes").groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas()

,product_name,product_id,price,sell_cnt,PU,revenue,ARPPU
0,Victory Dance - LEENALCHI x JINJO CREW,itemdesc.16100250,600.0,11647,11647,6988200,600.0
1,Victory Dance 49,itemdesc.16100259,500.0,8316,8316,4158000,500.0
2,Victory Dance 44,itemdesc.16100254,400.0,7821,7821,3128400,400.0
3,Victory Dance 41,itemdesc.16100141,500.0,6675,6675,3337500,500.0
4,Victory Dance 47,itemdesc.16100257,500.0,6341,6341,3170500,500.0
5,Victory Dance 46,itemdesc.16100256,500.0,5297,5297,2648500,500.0
6,Victory Dance 45,itemdesc.16100255,500.0,5247,5247,2623500,500.0
7,Victory Dance 43,itemdesc.16100253,500.0,3913,3913,1956500,500.0
8,Victory Dance 39,itemdesc.16100129,400.0,3704,3704,1481600,400.0
9,Victory Dance 40,itemdesc.16100249,500.0,3689,3689,1844500,500.0


#### 해당 기간동안 다른 상품들과 비교

In [88]:
mar2_period_gcoin_pc.groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas().head(20)

,product_name,product_id,price,sell_cnt,PU,revenue,ARPPU
0,GEAR HEAD I - CONTRABAND CRATE,itemdesc.14100006,885.106133,2809069,506283,2486324200,4910.937559
1,PAJAMA PARTY PASS & WEAPON PACK,itemdesc.13000258,990.000000,350707,350706,347199930,990.002823
2,Gold Plate - Beryl M762,itemdesc.12010408,990.000000,174385,174383,172641150,990.011354
3,Nickname Change,itemdesc.45000003,990.000000,126374,104881,125110260,1192.878214
4,Victory Dance - LEENALCHI x JINJO CREW,itemdesc.16100250,600.000000,121313,121313,72787800,600.000000
5,Legend Status Jersey,itemdesc.11010326,200.000000,110687,110683,22137400,200.007228
6,WEAPON SKIN KEY,itemdesc.15100002,250.000000,107961,43692,26990250,617.738945
7,5 Levels,itemdesc.45000150,500.000000,93758,66133,46879000,708.859420
8,Naval Officer Formal Gloves,itemdesc.11040032,100.000000,90873,90873,9087300,100.000000
9,Legend Basketball Jersey,itemdesc.11010330,200.000000,72966,72961,14593200,200.013706


In [89]:
mar2_period_gcoin_console.groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas().head(20)

,product_name,product_id,price,sell_cnt,PU,revenue,ARPPU
0,GEAR HEAD I - CONTRABAND CRATE,itemdesc.14100006,499.049988,75578,25776,37717200,1463.268156
1,PAJAMA PARTY PASS & WEAPON PACK,itemdesc.13000258,990.000000,20216,20216,20013840,990.000000
2,One Love Beanie,itemdesc.11060109,200.000000,12672,12672,2534400,200.000000
3,Victory Dance - LEENALCHI x JINJO CREW,itemdesc.16100250,600.000000,11647,11647,6988200,600.000000
4,Victory Dance 49,itemdesc.16100259,500.000000,8316,8316,4158000,500.000000
5,Victory Dance 44,itemdesc.16100254,400.000000,7821,7821,3128400,400.000000
6,Legend Status Jersey,itemdesc.11010326,200.000000,6856,6856,1371200,200.000000
7,Victory Dance 41,itemdesc.16100141,500.000000,6675,6675,3337500,500.000000
8,Victory Dance 47,itemdesc.16100257,500.000000,6341,6341,3170500,500.000000
9,Colorful Chaos Mask,itemdesc.11070073,350.000000,6063,6063,2122050,350.000000


### 콜라보 전후 변화

In [83]:
pc_start_date = "2021-03-06"
pc_end_date = "2021-03-19"

console_start_date = "2021-03-11"
console_end_date = "2021-03-24"

pre_post_mar2_user_pc = pre_load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='user_master').where(col("lastlogindate") >= pc_start_date)
pre_post_mar2_gcoin_pc = pre_load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use')
print('== PC DONE ==')

pre_post_mar2_user_console = pre_load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='user_master').where(col("lastlogindate") >= console_start_date)
pre_post_mar2_gcoin_console = pre_load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use')
print('== CONSOLE DONE ==')

== PC DONE ==
== CONSOLE DONE ==


PC

In [84]:
a = pre_post_mar2_user_pc.groupBy("date").agg(countDistinct("accountid").alias("au"))
b = pre_post_mar2_gcoin_pc.groupBy("date").agg(countDistinct("account_id").alias("pu"), sum("price").alias("revenue"), count("*").alias("cnt"))
c = a.join(b, "date", "left").withColumn("pur", col("pu")/col("au")).withColumn("arpu", col("revenue")/col("au")).withColumn("arppu", col("revenue")/col("pu"))

In [85]:
c.orderBy("date").toPandas()

,date,au,pu,revenue,cnt,pur,arpu,arppu
0,2021-03-06,1838950,30640,38191330,45729,0.016662,20.768009,1246.453329
1,2021-03-07,2578312,27982,34642490,41773,0.010853,13.436112,1238.027661
2,2021-03-08,2984390,21713,27337740,32689,0.007276,9.160244,1259.049417
3,2021-03-09,3299406,22227,27655980,32954,0.006737,8.382109,1244.251586
4,2021-03-10,3505424,63622,111432580,105445,0.018150,31.788617,1751.478734
5,2021-03-11,3745322,44684,67632310,69312,0.011931,18.057809,1513.568839
6,2021-03-12,4007826,43060,63410870,66209,0.010744,15.821762,1472.616582
7,2021-03-13,4303762,49896,63462180,74128,0.011594,14.745746,1271.889129
8,2021-03-14,4537563,39579,49533540,59565,0.008723,10.916331,1251.510650
9,2021-03-15,4696360,31844,41357020,48596,0.006781,8.806186,1298.738224


CONSOLE

In [86]:
a = pre_post_mar2_user_console.groupBy("date").agg(countDistinct("accountid").alias("au"))
b = pre_post_mar2_gcoin_console.groupBy("date").agg(countDistinct("account_id").alias("pu"), sum("price").alias("revenue"), count("*").alias("cnt"))
c = a.join(b, "date", "left").withColumn("pur", col("pu")/col("au")).withColumn("arpu", col("revenue")/col("au")).withColumn("arppu", col("revenue")/col("pu"))

In [87]:
c.orderBy("date").toPandas()

,date,au,pu,revenue,cnt,pur,arpu,arppu
0,2021-03-11,172155,1980,2364600,2992,0.011501,13.735297,1194.242424
1,2021-03-12,247343,2210,2637340,3400,0.008935,10.662683,1193.366516
2,2021-03-13,318216,2657,3322870,4299,0.008350,10.442184,1250.609710
3,2021-03-14,377141,2679,3178820,4107,0.007103,8.428731,1186.569616
4,2021-03-15,416244,2107,2480490,3252,0.005062,5.959221,1177.261509
5,2021-03-16,450859,2209,2635110,3430,0.004900,5.844643,1192.897239
6,2021-03-17,480609,1969,2450630,3120,0.004097,5.099010,1244.606399
7,2021-03-18,503860,4730,6417510,7930,0.009388,12.736693,1356.767442
8,2021-03-19,531687,4257,5591150,6774,0.008007,10.515867,1313.401456
9,2021-03-20,565477,3725,4769270,5948,0.006587,8.434065,1280.340940


## Elsolitario

In [7]:
item_meta_sales.where(col("event_name") == "202106_elsolitario").select("product_id").distinct().toPandas()["product_id"].tolist()

['itemdesc.13000290',
 'itemdesc.12031076',
 'itemdesc.13000291',
 'itemdesc.11040055',
 'itemdesc.12031077',
 'itemdesc.11030201',
 'itemdesc.11010354',
 'itemdesc.11030202',
 'itemdesc.12020032',
 'itemdesc.11010351',
 'itemdesc.11040057']

In [7]:
# elso_daily = all_economy_gcoin.where(col("product_id").isin(
#     item_meta_sales.where(col("event_name") == "202106_elsolitario").select("product_id").distinct().toPandas()["product_id"].tolist()
# )).toPandas()
# elso_daily.to_csv(DATA_PATH + 'elso_daily.csv', index=False)
elso_daily = pd.read_csv(DATA_PATH + 'elso_daily.csv')

In [218]:
elso_daily.groupby(["product_id", "pu_type"])[["pu", "unit_sold"]].sum()

pu  unit_sold
product_id        pu_type                              
itemdesc.11010351 Inactive               0.0        0.0
                  None                   0.0        0.0
                  existing           20026.0    20026.0
                  new                 3539.0     3539.0
                  return_long_term    1384.0     1384.0
                  return_short_term   3253.0     3253.0
itemdesc.11010354 Inactive               0.0        0.0
                  None                   0.0        0.0
                  existing            3589.0     3589.0
                  new                  866.0      866.0
                  return_long_term     369.0      369.0
                  return_short_term    600.0      600.0
itemdesc.11030201 Inactive               0.0        0.0
                  None                   0.0        0.0
                  existing             376.0      376.0
                  new                  141.0      141.0
                  return_long_term      48.0       48.0
                  return_short_term     65.0       65.0
itemdesc.11030202 Inactive               0.0        0.0
                  None                   0.0        0.0
                  existing            1168.0     1168.0
                  new                  735.0      735.0
                  return_long_term     322.0      322.0
                  return_short_term    203.0      203.0
itemdesc.11040055 Inactive               0.0        0.0
                  None                   0.0        0.0
                  existing           21108.0    21108.0
                  new                 4044.0     4044.0
                  return_long_term    1432.0     1432.0
                  return_short_term   3190.0     3190.0
...                                      ...        ...
itemdesc.12020032 Inactive               0.0        0.0
                  None                   0.0        0.0
                  existing           15528.0    15528.0
                  new                  891.0      891.0
                  return_long_term     804.0      804.0
                  return_short_term   2000.0     2000.0
itemdesc.12031076 Inactive               0.0        0.0
                  None                   0.0        0.0
                  existing            3182.0     3182.0
                  new                  704.0      704.0
                  return_long_term     341.0      341.0
                  return_short_term    467.0      467.0
itemdesc.12031077 Inactive               0.0        0.0
                  None                   0.0        0.0
                  existing           13688.0    13688.0
                  new                 5248.0     5248.0
                  return_long_term    2700.0     2700.0
                  return_short_term   2457.0     2457.0
itemdesc.13000290 Inactive               0.0        0.0
                  None                   0.0        0.0
                  existing            7162.0     7162.0
                  new                 1813.0     1813.0
                  return_long_term     366.0      366.0
                  return_short_term    801.0      801.0
itemdesc.13000291 Inactive               0.0        0.0
                  None                   0.0        0.0
                  existing            4074.0     4074.0
                  new                 1561.0     1561.0
                  return_long_term     371.0      371.0
                  return_short_term    502.0      502.0

[66 rows x 2 columns]

In [92]:
pc_start_date = "2021-06-02"
pc_end_date = "2021-10-22"

console_start_date = "2021-06-10"
console_end_date = "2021-10-22"

elso_user_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='user_master').where(col("lastlogindate") >= pc_start_date)
elso_gcoin_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use').where(col("event_name") == '202106_elsolitario')
print('== PC DONE ==')

elso_user_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='user_master').where(col("lastlogindate") >= console_start_date)
elso_gcoin_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use').where(col("event_name") == '202106_elsolitario')
print('== CONSOLE DONE ==')

== PC DONE ==
== CONSOLE DONE ==


In [117]:
elso_period_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use')
elso_period_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use')

In [133]:
elso_gcoin_pc_tmp = elso_gcoin_pc.join(elso_user_pc.select("country_os", "country_ip", "daily_country_os", "daily_country_ip", "accountid", "date"), (elso_gcoin_pc.account_id == elso_user_pc.accountid) & (elso_gcoin_pc.date == elso_user_pc.date), "left").select("*").toPandas()
elso_gcoin_pc_tmp.to_csv(DATA_PATH + "elso_gcoin_pc_tmp.csv", index=False)


In [134]:
elso_gcoin_console_tmp = elso_gcoin_console.join(elso_user_console.select("country_os", "country_ip", "daily_country_os", "daily_country_ip", "accountid", "date"), (elso_gcoin_console.account_id == elso_user_console.accountid) & (elso_gcoin_console.date == elso_user_console.date), "left").select("*").toPandas()
elso_gcoin_console_tmp.to_csv(DATA_PATH + "elso_gcoin_console_tmp.csv", index=False)

In [97]:
elso_pc_a = elso_user_pc.groupBy('platform').agg(countDistinct('accountid').alias('au')).toPandas()
elso_pc_b = elso_gcoin_pc.groupBy("platform").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()
elso_pc = pd.merge(elso_pc_a, elso_pc_b, on='platform', how="left").fillna(0)
elso_pc['pur'] = elso_pc.pu / elso_pc.au
elso_pc['arpu'] = elso_pc.revenue / elso_pc.au
elso_pc['arppu'] = elso_pc.revenue / elso_pc.pu

In [98]:
elso_pc.to_csv(DATA_PATH + "elso_pc.csv", index=False)

In [99]:
elso_pc

,platform,au,cnt,pu,revenue,pur,arpu,arppu
0,KAKAO,1522598,9976.0,8414.0,9469290.0,0.005526,6.219166,1125.420727
1,STEAM,14329503,145687.0,120594.0,90077010.0,0.008416,6.286122,746.944375
2,MAILRU,12904,29.0,29.0,15000.0,0.002247,1.162430,517.241379
3,DMM,2,0.0,0.0,0.0,0.000000,0.000000,NaN


In [101]:
elso_console_a = elso_user_console.groupBy('platform').agg(countDistinct('accountid').alias('au')).toPandas()
elso_console_b = elso_gcoin_console.groupBy("platform").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()
elso_console = pd.merge(elso_console_a, elso_console_b, on='platform', how="left").fillna(0)
elso_console['pur'] = elso_console.pu / elso_console.au
elso_console['arpu'] = elso_console.revenue / elso_console.au
elso_console['arppu'] = elso_console.revenue / elso_console.pu

In [102]:
elso_console.to_csv(DATA_PATH + "elso_console.csv", index=False)

In [103]:
elso_console

,platform,au,cnt,pu,revenue,pur,arpu,arppu
0,XBOX,1745726,10309,8312,6750580,0.004761,3.866918,812.148701
1,STADIA,37522,46,36,31200,0.000959,0.831512,866.666667
2,PSN,900558,4273,3470,2788780,0.003853,3.096724,803.682997


### 상품별

In [104]:
elso_product_gcoin_pc = elso_gcoin_pc.groupBy("product_name", "product_id").agg(countDistinct("transaction_id").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).withColumn("ARPPU", col("revenue")/col("pu")).orderBy("cnt", ascending=False).toPandas()

In [109]:
elso_product_gcoin_pc_platform = elso_gcoin_pc.groupBy("platform", "product_name", "product_id").agg(countDistinct("transaction_id").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).withColumn("ARPPU", col("revenue")/col("pu")).orderBy("platform", "cnt", ascending=False).toPandas()

In [110]:
elso_product_gcoin_pc.to_csv(DATA_PATH + "elso_product_gcoin_pc.csv", index=False)
elso_product_gcoin_pc_platform.to_csv(DATA_PATH + "elso_product_gcoin_pc_platform.csv", index=False)

In [107]:
elso_product_gcoin_pc

,product_name,product_id,cnt,pu,revenue,ARPPU
0,El Solitario Wolf Gloves,itemdesc.11040057,47004,47003,14101200,300.006383
1,El Solitario Biker Gloves,itemdesc.11040055,28329,28329,8498700,300.000000
2,El Solitario Biker Suit,itemdesc.11010351,26039,26039,25778610,990.000000
3,El Solitario Wolf - Helmet (Level 1),itemdesc.12031077,16452,16452,8226000,500.000000
4,El Solitario Desert Wolves,itemdesc.12020032,16387,16387,16223130,990.000000
5,EL SOLITARIO BIKER SET,itemdesc.13000290,9409,9409,14583950,1550.000000
6,EL SOLITARIO WOLF SET,itemdesc.13000291,4516,4516,6999800,1550.000000
7,El Solitario Snake Head - Helmet (Level 1),itemdesc.12031076,3448,3448,1724000,500.000000
8,El Solitario Wolf Bodysuit,itemdesc.11010354,3179,3179,3147210,990.000000
9,El Solitario Biker Boots (Gray),itemdesc.11030202,699,699,209700,300.000000


In [111]:
elso_product_gcoin_pc_platform

,platform,product_name,product_id,cnt,pu,revenue,ARPPU
0,STEAM,El Solitario Wolf Gloves,itemdesc.11040057,46267,46266,13880100,300.006484
1,STEAM,El Solitario Biker Gloves,itemdesc.11040055,28027,28027,8408100,300.000000
2,STEAM,El Solitario Biker Suit,itemdesc.11010351,22209,22209,21986910,990.000000
3,STEAM,El Solitario Desert Wolves,itemdesc.12020032,15341,15341,15187590,990.000000
4,STEAM,El Solitario Wolf - Helmet (Level 1),itemdesc.12031077,15075,15075,7537500,500.000000
5,STEAM,EL SOLITARIO BIKER SET,itemdesc.13000290,8109,8109,12568950,1550.000000
6,STEAM,EL SOLITARIO WOLF SET,itemdesc.13000291,3802,3802,5893100,1550.000000
7,STEAM,El Solitario Snake Head - Helmet (Level 1),itemdesc.12031076,3287,3287,1643500,500.000000
8,STEAM,El Solitario Wolf Bodysuit,itemdesc.11010354,2754,2754,2726460,990.000000
9,STEAM,El Solitario Biker Boots (Gray),itemdesc.11030202,621,621,186300,300.000000


In [112]:
elso_product_gcoin_console = elso_gcoin_console.groupBy("product_name", "product_id").agg(countDistinct("transaction_id").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).withColumn("ARPPU", col("revenue")/col("pu")).orderBy("cnt", ascending=False).toPandas()

In [113]:
elso_product_gcoin_console_platform = elso_gcoin_console.groupBy("platform", "product_name", "product_id").agg(countDistinct("transaction_id").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).withColumn("ARPPU", col("revenue")/col("pu")).orderBy("platform", "cnt", ascending=False).toPandas()

In [114]:
elso_product_gcoin_console.to_csv(DATA_PATH + "elso_product_gcoin_console.csv", index=False)
elso_product_gcoin_console_platform.to_csv(DATA_PATH + "elso_product_gcoin_console_platform.csv", index=False)

In [115]:
elso_product_gcoin_console

,product_name,product_id,cnt,pu,revenue,ARPPU
0,El Solitario Wolf - Helmet (Level 1),itemdesc.12031077,3876,3876,1938000,500.0
1,El Solitario Wolf Gloves,itemdesc.11040057,3176,3176,952800,300.0
2,El Solitario Desert Wolves,itemdesc.12020032,1444,1444,1429560,990.0
3,El Solitario Biker Suit,itemdesc.11010351,1135,1135,1123650,990.0
4,El Solitario Wolf Bodysuit,itemdesc.11010354,1135,1135,1123650,990.0
5,EL SOLITARIO WOLF SET,itemdesc.13000291,1002,1002,1553100,1550.0
6,El Solitario Biker Boots (Gray),itemdesc.11030202,867,867,260100,300.0
7,El Solitario Biker Gloves,itemdesc.11040055,786,786,235800,300.0
8,El Solitario Snake Head - Helmet (Level 1),itemdesc.12031076,634,634,317000,500.0
9,EL SOLITARIO BIKER SET,itemdesc.13000290,372,372,576600,1550.0


In [116]:
elso_product_gcoin_console_platform

,platform,product_name,product_id,cnt,pu,revenue,ARPPU
0,XBOX,El Solitario Wolf - Helmet (Level 1),itemdesc.12031077,2873,2873,1436500,500.0
1,XBOX,El Solitario Wolf Gloves,itemdesc.11040057,2113,2113,633900,300.0
2,XBOX,El Solitario Desert Wolves,itemdesc.12020032,1047,1047,1036530,990.0
3,XBOX,El Solitario Biker Suit,itemdesc.11010351,791,791,783090,990.0
4,XBOX,El Solitario Wolf Bodysuit,itemdesc.11010354,769,769,761310,990.0
5,XBOX,EL SOLITARIO WOLF SET,itemdesc.13000291,694,694,1075700,1550.0
6,XBOX,El Solitario Biker Boots (Gray),itemdesc.11030202,581,581,174300,300.0
7,XBOX,El Solitario Biker Gloves,itemdesc.11040055,545,545,163500,300.0
8,XBOX,El Solitario Snake Head - Helmet (Level 1),itemdesc.12031076,491,491,245500,500.0
9,XBOX,EL SOLITARIO BIKER SET,itemdesc.13000290,255,255,395250,1550.0


### 해당 기간동안 다른 상품들과 비교

In [119]:
elso_period_product_rank_pc = elso_period_pc.groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas()

In [120]:
elso_period_product_rank_pc.to_csv(DATA_PATH + "elso_period_product_rank_pc.csv", index=False)

In [121]:
elso_period_product_rank_console = elso_period_console.groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas()

In [122]:
elso_period_product_rank_console.to_csv(DATA_PATH + "elso_period_product_rank_console.csv", index=False)

### 콜라보 시작 전 후 1주일

In [124]:
pc_start_date = "2021-05-26"
pc_end_date = "2021-06-08"

console_start_date = "2021-06-03"
console_end_date = "2021-06-16"

pre_post_elso_user_pc = pre_load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='user_master').where(col("lastlogindate") >= pc_start_date)
pre_post_elso_gcoin_pc = pre_load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use')
print('== PC DONE ==')

pre_post_elso_user_console = pre_load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='user_master').where(col("lastlogindate") >= console_start_date)
pre_post_elso_gcoin_console = pre_load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use')
print('== CONSOLE DONE ==')

== PC DONE ==
== CONSOLE DONE ==


PC

In [125]:
a = pre_post_elso_user_pc.groupBy("date").agg(countDistinct("accountid").alias("au"))
b = pre_post_elso_gcoin_pc.groupBy("date").agg(countDistinct("account_id").alias("pu"), sum("price").alias("revenue"), count("*").alias("cnt"))
c = a.join(b, "date", "left").withColumn("pur", col("pu")/col("au")).withColumn("arpu", col("revenue")/col("au")).withColumn("arppu", col("revenue")/col("pu"))

In [126]:
c.orderBy("date").toPandas()

,date,au,pu,revenue,cnt,pur,arpu,arppu
0,2021-05-26,1516276,44028,48666260,70449,0.029037,32.095911,1105.347960
1,2021-05-27,2125097,33307,36405360,51996,0.015673,17.131152,1093.024289
2,2021-05-28,2624148,30530,33950100,47708,0.011634,12.937571,1112.024238
3,2021-05-29,3116090,33610,36777780,51995,0.010786,11.802541,1094.251116
4,2021-05-30,3478113,32142,34366780,49366,0.009241,9.880869,1069.217224
5,2021-05-31,3704143,26406,29584700,41453,0.007129,7.986922,1120.377944
6,2021-06-01,3905279,26738,29718290,41629,0.006847,7.609774,1111.462712
7,2021-06-02,4061537,166980,553982890,640585,0.041113,136.397352,3317.660139
8,2021-06-03,4244417,144434,496581550,557961,0.034029,116.996410,3438.120872
9,2021-06-04,4435250,101841,345466120,384203,0.022962,77.891014,3392.210603


CONSOLE

In [127]:
a = pre_post_elso_user_console.groupBy("date").agg(countDistinct("accountid").alias("au"))
b = pre_post_elso_gcoin_console.groupBy("date").agg(countDistinct("account_id").alias("pu"), sum("price").alias("revenue"), count("*").alias("cnt"))
c = a.join(b, "date", "left").withColumn("pur", col("pu")/col("au")).withColumn("arpu", col("revenue")/col("au")).withColumn("arppu", col("revenue")/col("pu"))

In [128]:
c.orderBy("date").toPandas()

,date,au,pu,revenue,cnt,pur,arpu,arppu
0,2021-06-03,153281,2920,3300040,5146,0.019050,21.529348,1130.150685
1,2021-06-04,227061,2799,3059640,4808,0.012327,13.474969,1093.118971
2,2021-06-05,290748,2840,3045370,4914,0.009768,10.474259,1072.313380
3,2021-06-06,345154,2627,2660720,4266,0.007611,7.708791,1012.835935
4,2021-06-07,382871,1830,1799390,2990,0.004780,4.699729,983.273224
5,2021-06-08,416155,1592,1586840,2647,0.003825,3.813098,996.758794
6,2021-06-09,444936,1447,1419530,2339,0.003252,3.190414,981.015895
7,2021-06-10,467055,9370,15966190,26130,0.020062,34.184818,1703.969050
8,2021-06-11,493192,8401,12032240,21126,0.017034,24.396665,1432.239019
9,2021-06-12,523731,6707,8833580,16066,0.012806,16.866636,1317.068734


## Phao

In [129]:
# phao_daily = all_economy_gcoin.where(col("product_id").isin(
#     item_meta_sales.where(col("event_name") == "202107_phao_emote").select("product_id").distinct().toPandas()["product_id"].tolist()
# )).toPandas()
# phao_daily.to_csv(DATA_PATH + 'phao_daily.csv', index=False)
phao_daily = pd.read_csv(DATA_PATH + 'phao_daily.csv')

In [105]:
phao_daily = all_economy_gcoin.where(col("product_id").isin(
    item_meta_sales.where(col("event_name") == "202107_phao_emote").select("product_id").distinct().toPandas()["product_id"].tolist()
)).toPandas()

In [107]:
item_meta_sales.where(col("event_name") == "202107_phao_emote").select("product_id").distinct().toPandas()["product_id"].tolist()

['itemdesc.16100269']

In [8]:
pc_start_date = "2021-07-07"
pc_end_date = "2021-10-06"

console_start_date = "2021-07-15"
console_end_date = "2021-10-14"

event_name = "202107_phao_emote"
phao_user_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='user_master').where(col("lastlogindate") >= pc_start_date)
phao_gcoin_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use').where(col("event_name") == event_name)
print('== PC DONE ==')

phao_user_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='user_master').where(col("lastlogindate") >= console_start_date)
phao_gcoin_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use').where(col("event_name") == event_name)
print('== CONSOLE DONE ==')

== PC DONE ==
== CONSOLE DONE ==


In [17]:
phao_period_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use')
phao_period_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use')

In [136]:
phao_gcoin_pc_tmp = phao_gcoin_pc.join(phao_user_pc.select("country_os", "country_ip", "daily_country_os", "daily_country_ip", "accountid", "date"), (phao_gcoin_pc.account_id == phao_user_pc.accountid) & (phao_gcoin_pc.date == phao_user_pc.date), "left").select("*").toPandas()
phao_gcoin_pc_tmp.to_csv(DATA_PATH + "phao_gcoin_pc_tmp.csv", index=False)


In [137]:
phao_gcoin_console_tmp = phao_gcoin_console.join(phao_user_console.select("country_os", "country_ip", "daily_country_os", "daily_country_ip", "accountid", "date"), (phao_gcoin_console.account_id == phao_user_console.accountid) & (phao_gcoin_console.date == phao_user_console.date), "left").select("*").toPandas()
phao_gcoin_console_tmp.to_csv(DATA_PATH + "phao_gcoin_console_tmp.csv", index=False)


In [138]:
phao_pc_a = phao_user_pc.groupBy('platform').agg(countDistinct('accountid').alias('au')).toPandas()
phao_pc_b = phao_gcoin_pc.groupBy("platform").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()
phao_pc = pd.merge(phao_pc_a, phao_pc_b, on='platform', how="left").fillna(0)
phao_pc['pur'] = phao_pc.pu / phao_pc.au
phao_pc['arpu'] = phao_pc.revenue / phao_pc.au
phao_pc['arppu'] = phao_pc.revenue / phao_pc.pu

In [139]:
phao_pc.to_csv(DATA_PATH + "phao_pc.csv", index=False)

In [140]:
phao_pc

,platform,au,cnt,pu,revenue,pur,arpu,arppu
0,KAKAO,1215362,15944,15944,9566400,0.013119,7.871235,600.0
1,STEAM,11980105,125065,125065,75039000,0.010439,6.263635,600.0
2,MAILRU,10367,33,33,19800,0.003183,1.909906,600.0


In [141]:
phao_console_a = phao_user_console.groupBy('platform').agg(countDistinct('accountid').alias('au')).toPandas()
phao_console_b = phao_gcoin_console.groupBy("platform").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()
phao_console = pd.merge(phao_console_a, phao_console_b, on='platform', how="left").fillna(0)
phao_console['pur'] = phao_console.pu / phao_console.au
phao_console['arpu'] = phao_console.revenue / phao_console.au
phao_console['arppu'] = phao_console.revenue / phao_console.pu

In [142]:
phao_console.to_csv(DATA_PATH + "phao_console.csv", index=False)

In [143]:
phao_console

,platform,au,cnt,pu,revenue,pur,arpu,arppu
0,XBOX,1374665,12113,12113,7267800,0.008812,5.286961,600.0
1,STADIA,21072,45,45,27000,0.002136,1.281321,600.0
2,PSN,686814,4890,4890,2934000,0.007120,4.271899,600.0


In [22]:
phao_product_gcoin_pc

,product_name,product_id,cnt,pu,revenue,ARPPU
0,Victory Dance - 2 Phut Hon (Kaiz Remix),itemdesc.13000312,141042,141042,84625200,600.0


In [18]:
phao_period_pc.where(col("sub_category") == "emotes").groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas()

,product_name,product_id,price,sell_cnt,PU,revenue,ARPPU
0,Victory Dance - 2 Phut Hon (Kaiz Remix),itemdesc.13000312,600.0,141042,141042,84625200,600.000000
1,Victory Dance 62,itemdesc.16100279,600.0,131536,131536,78921600,600.000000
2,Victory Dance 58,itemdesc.16100272,500.0,30830,30830,15415000,500.000000
3,Victory Dance 57,itemdesc.16100270,500.0,29316,29316,14658000,500.000000
4,Victory Dance 65,itemdesc.16100286,500.0,27434,27434,13717000,500.000000
5,Victory Dance 61,itemdesc.16100278,500.0,24940,24940,12470000,500.000000
6,Victory Dance 63,itemdesc.16100284,500.0,22634,22634,11317000,500.000000
7,Victory Dance 66,itemdesc.16100287,500.0,21883,21883,10941500,500.000000
8,Victory Dance 39,itemdesc.16100129,400.0,21385,21385,8554000,400.000000
9,Victory Dance 41,itemdesc.16100141,500.0,19109,19109,9554500,500.000000


In [33]:
phao_period_console.where(col("sub_category") == "crate").groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas()

,product_name,product_id,price,sell_cnt,PU,revenue,ARPPU
0,CATACLYSMIC MAGMA - CONTRABAND CRATE,itemdesc.14100007,526.771767,124706,38697,65691600,1697.588960
1,BUZZKILL - CONTRABAND CRATE,itemdesc.14100008,584.685606,80186,23800,46883600,1969.899160
2,GEAR HEAD I - CONTRABAND CRATE,itemdesc.14100006,551.623071,54249,17850,29925000,1676.470588
3,FRACTURED ELEMENT - CONTRABAND CRATE,itemdesc.14100009,542.362872,33527,11946,18183800,1522.166416
4,VICTORY DANCE - 2 PHUT HON (KAIZ REMIX),itemdesc.13000312,600.000000,17048,17048,10228800,600.000000
5,TRICK-OR-TREAT - CONTRABAND CRATE,itemdesc.14100010,861.856662,10395,2322,8959000,3858.311800
6,MOON RABBIT BUNDLE,itemdesc.13000350,1590.000000,3692,3692,5870280,1590.000000
7,PCS5 SQUAD WIPE SET,itemdesc.13000340,990.000000,3513,3513,3477870,990.000000
8,JISOO'S BLACKPINK BUNDLE,itemdesc.13000315,2780.000000,2698,2698,7500440,2780.000000
9,ROSÉ'S BLACKPINK BUNDLE,itemdesc.13000314,2780.000000,2635,2635,7325300,2780.000000


CONSOLE에서는 sub_category가 emotes가 아닌 crate로 들어가있음

In [23]:
phao_period_console.where(col("sub_category") == "emotes").groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas()

,product_name,product_id,price,sell_cnt,PU,revenue,ARPPU
0,Victory Dance 62,itemdesc.16100279,600.0,8911,8911,5346600,600.0
1,Victory Dance 58,itemdesc.16100272,500.0,7822,7822,3911000,500.0
2,Victory Dance 54,itemdesc.16100266,500.0,3333,3333,1666500,500.0
3,Victory Dance 49,itemdesc.16100259,500.0,3062,3062,1531000,500.0
4,PCS5 Dance - Winning with Style,itemdesc.16100290,500.0,2730,2730,1365000,500.0
5,Victory Dance 57,itemdesc.16100270,500.0,2287,2287,1143500,500.0
6,Victory Dance 65,itemdesc.16100286,500.0,2157,2157,1078500,500.0
7,Victory Dance 39,itemdesc.16100129,400.0,2080,2080,832000,400.0
8,Victory Dance 41,itemdesc.16100141,500.0,1986,1986,993000,500.0
9,Victory Dance 55,itemdesc.16100267,500.0,1973,1973,986500,500.0


In [19]:
phao_period_product_rank_pc = phao_period_pc.groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas()

In [20]:
phao_period_product_rank_pc.to_csv(DATA_PATH + "phao_period_product_rank_pc.csv", index=False)

In [21]:
phao_period_product_rank_pc.head(20)

,product_name,product_id,price,sell_cnt,PU,revenue,ARPPU
0,BUZZKILL - CONTRABAND CRATE,itemdesc.14100008,944.571183,2115737,328311,1998464200,6087.107042
1,CATACLYSMIC MAGMA - CONTRABAND CRATE,itemdesc.14100007,807.386474,1637642,363024,1322210000,3642.210983
2,GEAR HEAD I - CONTRABAND CRATE,itemdesc.14100006,915.258268,1431342,259352,1310047600,5051.233844
3,FRACTURED ELEMENT - CONTRABAND CRATE,itemdesc.14100009,686.531982,822415,237166,564614200,2380.670922
4,TRICK-OR-TREAT - CONTRABAND CRATE,itemdesc.14100010,998.254609,330012,83574,329436000,3941.847943
5,SURVIVOR PASS: TAEGO WEAPON PACK,itemdesc.13000302,990.000000,293631,293631,290694690,990.000000
6,SURVIVOR PASS: CRAZY NIGHT WEAPON PACK,itemdesc.13000337,990.000000,236116,236116,233754840,990.000000
7,TAEGO Level UP,battlepasslevelup.202107,100.000000,234348,147598,23434800,158.774509
8,None,itemdesc.13000312,600.000000,229589,229588,137753400,600.002613
9,Victory Dance - 2 Phut Hon (Kaiz Remix),itemdesc.13000312,600.000000,141042,141042,84625200,600.000000


In [24]:
phao_period_product_rank_console = phao_period_console.groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas()

In [25]:
phao_period_product_rank_console.to_csv(DATA_PATH + "phao_period_product_rank_console.csv", index=False)

In [27]:
phao_period_product_rank_console.head(20)

,product_name,product_id,price,sell_cnt,PU,revenue,ARPPU
0,CATACLYSMIC MAGMA - CONTRABAND CRATE,itemdesc.14100007,526.771767,124706,38697,65691600,1697.588960
1,BUZZKILL - CONTRABAND CRATE,itemdesc.14100008,584.685606,80186,23800,46883600,1969.899160
2,GEAR HEAD I - CONTRABAND CRATE,itemdesc.14100006,551.623071,54249,17850,29925000,1676.470588
3,FRACTURED ELEMENT - CONTRABAND CRATE,itemdesc.14100009,542.362872,33527,11946,18183800,1522.166416
4,SURVIVOR PASS: TAEGO WEAPON PACK,itemdesc.13000302,990.000000,29029,29029,28738710,990.000000
5,SURVIVOR PASS: CRAZY NIGHT WEAPON PACK,itemdesc.13000337,990.000000,18719,18719,18531810,990.000000
6,VICTORY DANCE - 2 PHUT HON (KAIZ REMIX),itemdesc.13000312,600.000000,17048,17048,10228800,600.000000
7,None,battlepasslevelup.202107,100.000000,15392,9628,1539200,159.867054
8,TRICK-OR-TREAT - CONTRABAND CRATE,itemdesc.14100010,861.856662,10395,2322,8959000,3858.311800
9,Victory Dance 62,itemdesc.16100279,600.000000,8911,8911,5346600,600.000000


In [37]:
pc_start_date = "2021-06-30"
pc_end_date = "2021-07-13"

console_start_date = "2021-07-08"
console_end_date = "2021-07-21"

pre_post_phao_user_pc = pre_load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='user_master').where(col("lastlogindate") >= pc_start_date)
pre_post_phao_gcoin_pc = pre_load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use')
print('== PC DONE ==')

pre_post_phao_user_console = pre_load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='user_master').where(col("lastlogindate") >= console_start_date)
pre_post_phao_gcoin_console = pre_load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use')
print('== CONSOLE DONE ==')

== PC DONE ==
== CONSOLE DONE ==


PC

In [38]:
a = pre_post_phao_user_pc.groupBy("date").agg(countDistinct("accountid").alias("au"))
b = pre_post_phao_gcoin_pc.groupBy("date").agg(countDistinct("account_id").alias("pu"), sum("price").alias("revenue"), count("*").alias("cnt"))
c = a.join(b, "date", "left").withColumn("pur", col("pu")/col("au")).withColumn("arpu", col("revenue")/col("au")).withColumn("arppu", col("revenue")/col("pu"))

In [39]:
c.orderBy("date").toPandas()

,date,au,pu,revenue,cnt,pur,arpu,arppu
0,2021-06-30,1570302,41833,103233860,105819,0.026640,65.741405,2467.761337
1,2021-07-01,2276034,28364,70840430,77455,0.012462,31.124504,2497.547243
2,2021-07-02,2851647,28770,72417870,80187,0.010089,25.395103,2517.131387
3,2021-07-03,3390299,29149,71224540,80766,0.008598,21.008336,2443.464270
4,2021-07-04,3803798,26505,58902820,70512,0.006968,15.485265,2222.328617
5,2021-07-05,4076508,20186,49221500,57245,0.004952,12.074427,2438.397900
6,2021-07-06,4326314,19094,46254850,53598,0.004413,10.691515,2422.480884
7,2021-07-07,4535160,175102,320967600,336581,0.038610,70.773159,1833.032176
8,2021-07-08,4761941,118692,177748890,231982,0.024925,37.326983,1497.564200
9,2021-07-09,4987038,89613,128893530,180645,0.017969,25.845708,1438.335175


CONSOLE

In [40]:
a = pre_post_phao_user_console.groupBy("date").agg(countDistinct("accountid").alias("au"))
b = pre_post_phao_gcoin_console.groupBy("date").agg(countDistinct("account_id").alias("pu"), sum("price").alias("revenue"), count("*").alias("cnt"))
c = a.join(b, "date", "left").withColumn("pur", col("pu")/col("au")).withColumn("arpu", col("revenue")/col("au")).withColumn("arppu", col("revenue")/col("pu"))

In [41]:
c.orderBy("date").toPandas()

,date,au,pu,revenue,cnt,pur,arpu,arppu
0,2021-07-08,159498,2921,4189770,6128,0.018314,26.268480,1434.361520
1,2021-07-09,230045,2975,4189970,6419,0.012932,18.213697,1408.393277
2,2021-07-10,294421,3112,3980070,6504,0.010570,13.518295,1278.942802
3,2021-07-11,346545,2929,3581430,6024,0.008452,10.334675,1222.748378
4,2021-07-12,387259,2372,3321950,5184,0.006125,8.578109,1400.484823
5,2021-07-13,423612,2191,2850800,4683,0.005172,6.729743,1301.141031
6,2021-07-14,457711,2050,2616870,4500,0.004479,5.717298,1276.521951
7,2021-07-15,486980,13973,24144620,27923,0.028693,49.580311,1727.948186
8,2021-07-16,523588,11251,16555970,22008,0.021488,31.620224,1471.510977
9,2021-07-17,562087,8292,11422580,16603,0.014752,20.321730,1377.542209


## Don Lee 
- Survivor Pass : TAEGO 판매 증대
- 2021-07-07 11:00:00 +09:00 ~ 2021-08-18 11:00:00 +09:00
- salesitemdesc.1550, salesitemdesc.1551                        

## Son

In [100]:
item_meta_sales.where(col("event_name") == "202107_son").select("product_id").distinct().toPandas()["product_id"].tolist()

['itemdesc.12010446',
 'itemdesc.13000310',
 'itemdesc.11040058',
 'itemdesc.12010445',
 'itemdesc.11020244',
 'itemdesc.11010364',
 'itemdesc.11030211',
 'itemdesc.11030212',
 'itemdesc.11050162',
 'itemdesc.13000308',
 'itemdesc.11020243',
 'itemdesc.11010365',
 'itemdesc.13000309',
 'itemdesc.13000311']

In [27]:
# son_daily = all_economy_gcoin.where(col("product_id").isin(
#     item_meta_sales.where(col("event_name") == "202107_son").select("product_id").distinct().toPandas()["product_id"].tolist()
# )).toPandas()
# son_daily.to_csv(DATA_PATH + 'son_daily.csv', index=False)
son_daily = pd.read_csv(DATA_PATH + 'son_daily.csv')

In [42]:
pc_start_date = "2021-07-21"
pc_end_date = "2021-10-20"

console_start_date = "2021-07-21"
console_end_date = "2021-10-20"

event_name = "202107_son"
son_user_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='user_master').where(col("lastlogindate") >= pc_start_date)
son_gcoin_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use').where(col("event_name") == event_name)
print('== PC DONE ==')

son_user_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='user_master').where(col("lastlogindate") >= console_start_date)
son_gcoin_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use').where(col("event_name") == event_name)
print('== CONSOLE DONE ==')

== PC DONE ==
== CONSOLE DONE ==


In [43]:
son_period_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use')
son_period_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use')

In [44]:
son_pc_a = son_user_pc.groupBy('platform').agg(countDistinct('accountid').alias('au')).toPandas()
son_pc_b = son_gcoin_pc.groupBy("platform").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()
son_pc = pd.merge(son_pc_a, son_pc_b, on='platform', how="left").fillna(0)
son_pc['pur'] = son_pc.pu / son_pc.au
son_pc['arpu'] = son_pc.revenue / son_pc.au
son_pc['arppu'] = son_pc.revenue / son_pc.pu

In [45]:
son_pc.to_csv(DATA_PATH + "son_pc.csv", index=False)

In [46]:
son_pc

,platform,au,cnt,pu,revenue,pur,arpu,arppu
0,KAKAO,1182666,16238,13233,25750130,0.011189,21.772952,1945.902668
1,STEAM,11522776,173144,147185,189023660,0.012773,16.404351,1284.258994
2,MAILRU,10454,47,39,43750,0.003731,4.185001,1121.794872


In [47]:
son_console_a = son_user_console.groupBy('platform').agg(countDistinct('accountid').alias('au')).toPandas()
son_console_b = son_gcoin_console.groupBy("platform").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()
son_console = pd.merge(son_console_a, son_console_b, on='platform', how="left").fillna(0)
son_console['pur'] = son_console.pu / son_console.au
son_console['arpu'] = son_console.revenue / son_console.au
son_console['arppu'] = son_console.revenue / son_console.pu

In [48]:
son_console.to_csv(DATA_PATH + "son_console.csv", index=False)

In [49]:
son_console

,platform,au,cnt,pu,revenue,pur,arpu,arppu
0,XBOX,1348213,16247,10985,13072380,0.008148,9.696079,1190.020938
1,STADIA,18910,66,41,41430,0.002168,2.190904,1010.487805
2,PSN,674104,7747,5215,6072190,0.007736,9.007794,1164.370086


### 상품별

In [50]:
son_product_gcoin_pc = son_gcoin_pc.groupBy("product_name", "product_id").agg(countDistinct("transaction_id").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).withColumn("ARPPU", col("revenue")/col("pu")).orderBy("cnt", ascending=False).toPandas()
son_product_gcoin_pc_platform = son_gcoin_pc.groupBy("platform", "product_name", "product_id").agg(countDistinct("transaction_id").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).withColumn("ARPPU", col("revenue")/col("pu")).orderBy("platform", "cnt", ascending=False).toPandas()

In [51]:
son_product_gcoin_pc.to_csv(DATA_PATH + "son_product_gcoin_pc.csv", index=False)
son_product_gcoin_pc_platform.to_csv(DATA_PATH + "son_product_gcoin_pc_platform.csv", index=False)

In [52]:
son_product_gcoin_console = son_gcoin_console.groupBy("product_name", "product_id").agg(countDistinct("transaction_id").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).withColumn("ARPPU", col("revenue")/col("pu")).orderBy("cnt", ascending=False).toPandas()
son_product_gcoin_console_platform = son_gcoin_console.groupBy("platform", "product_name", "product_id").agg(countDistinct("transaction_id").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).withColumn("ARPPU", col("revenue")/col("pu")).orderBy("platform", "cnt", ascending=False).toPandas()

In [53]:
son_product_gcoin_console.to_csv(DATA_PATH + "son_product_gcoin_console.csv", index=False)
son_product_gcoin_console_platform.to_csv(DATA_PATH + "son_product_gcoin_console_platform.csv", index=False)

PC

In [54]:
son_product_gcoin_pc

,product_name,product_id,cnt,pu,revenue,ARPPU
0,Son Heung-min - M416,itemdesc.12010445,56729,56729,56161710,990.0
1,SON HEUNG-MIN'S WEAPON BUNDLE,itemdesc.13000311,45972,45972,68958000,1500.0
2,Son Heung-min's Gloves,itemdesc.11040058,25352,25352,7605600,300.0
3,SON HEUNG-MIN'S OUTFIT BUNDLE,itemdesc.13000310,20624,20624,52178720,2530.0
4,Son Heung-min - Mini14,itemdesc.12010446,8278,8278,8195220,990.0
5,Son Heung-min's Shirt,itemdesc.11010364,7710,7710,3855000,500.0
6,Son Heung-min's Shoes,itemdesc.11030211,5617,5617,1685100,300.0
7,Son Heung-min's Jersey,itemdesc.11010365,5347,5347,2673500,500.0
8,SON HEUNG-MIN'S CLOTHING BUNDLE,itemdesc.13000309,3076,3076,5290720,1720.0
9,Son Heung-min's Slacks,itemdesc.11020243,2825,2825,1412500,500.0


In [55]:
son_product_gcoin_pc_platform

,platform,product_name,product_id,cnt,pu,revenue,ARPPU
0,STEAM,Son Heung-min - M416,itemdesc.12010445,54304,54304,53760960,990.0
1,STEAM,SON HEUNG-MIN'S WEAPON BUNDLE,itemdesc.13000311,42752,42752,64128000,1500.0
2,STEAM,Son Heung-min's Gloves,itemdesc.11040058,24899,24899,7469700,300.0
3,STEAM,SON HEUNG-MIN'S OUTFIT BUNDLE,itemdesc.13000310,14659,14659,37087270,2530.0
4,STEAM,Son Heung-min - Mini14,itemdesc.12010446,7658,7658,7581420,990.0
5,STEAM,Son Heung-min's Shirt,itemdesc.11010364,6570,6570,3285000,500.0
6,STEAM,Son Heung-min's Shoes,itemdesc.11030211,5190,5190,1557000,300.0
7,STEAM,Son Heung-min's Jersey,itemdesc.11010365,5083,5083,2541500,500.0
8,STEAM,SON HEUNG-MIN'S CLOTHING BUNDLE,itemdesc.13000309,2593,2593,4459960,1720.0
9,STEAM,Son Heung-min's Slacks,itemdesc.11020243,2402,2402,1201000,500.0


CONSOLE

In [56]:
son_product_gcoin_console

,product_name,product_id,cnt,pu,revenue,ARPPU
0,Son Heung-min's Shirt,itemdesc.11010364,4256,4256,2128000,500.0
1,Son Heung-min's Gloves,itemdesc.11040058,3789,3789,1136700,300.0
2,Son Heung-min - M416,itemdesc.12010445,2729,2729,2701710,990.0
3,Son Heung-min's Slacks,itemdesc.11020243,2481,2481,1240500,500.0
4,Son Heung-min's Jersey,itemdesc.11010365,2006,2006,1003000,500.0
5,Son Heung-min's Jacket,itemdesc.11050162,1572,1572,1556280,990.0
6,SON HEUNG-MIN'S WEAPON BUNDLE,itemdesc.13000311,1563,1563,2344500,1500.0
7,SON HEUNG-MIN'S CLOTHING BUNDLE,itemdesc.13000309,1337,1337,2299640,1720.0
8,SON HEUNG-MIN'S OUTFIT BUNDLE,itemdesc.13000310,1199,1199,3033470,2530.0
9,Son Heung-min's Shoes,itemdesc.11030211,1147,1147,344100,300.0


In [57]:
son_product_gcoin_console_platform

,platform,product_name,product_id,cnt,pu,revenue,ARPPU
0,XBOX,Son Heung-min's Shirt,itemdesc.11010364,2830,2830,1415000,500.0
1,XBOX,Son Heung-min's Gloves,itemdesc.11040058,2518,2518,755400,300.0
2,XBOX,Son Heung-min - M416,itemdesc.12010445,1906,1906,1886940,990.0
3,XBOX,Son Heung-min's Slacks,itemdesc.11020243,1612,1612,806000,500.0
4,XBOX,Son Heung-min's Jersey,itemdesc.11010365,1388,1388,694000,500.0
5,XBOX,SON HEUNG-MIN'S WEAPON BUNDLE,itemdesc.13000311,1132,1132,1698000,1500.0
6,XBOX,Son Heung-min's Jacket,itemdesc.11050162,1048,1048,1037520,990.0
7,XBOX,SON HEUNG-MIN'S CLOTHING BUNDLE,itemdesc.13000309,935,935,1608200,1720.0
8,XBOX,SON HEUNG-MIN'S OUTFIT BUNDLE,itemdesc.13000310,777,777,1965810,2530.0
9,XBOX,Son Heung-min's Shoes,itemdesc.11030211,740,740,222000,300.0


### 해당 기간 다른 상품들과 비교

PC

In [58]:
son_period_product_rank_pc = son_period_pc.groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas()

In [59]:
son_period_product_rank_pc.to_csv(DATA_PATH + "son_period_product_rank_pc.csv", index=False)

In [60]:
son_period_product_rank_console = son_period_console.groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas()

In [61]:
son_period_product_rank_console.to_csv(DATA_PATH + "son_period_product_rank_console.csv", index=False)

### 콜라보 전후 1주일

In [62]:
pc_start_date = "2021-07-14"
pc_end_date = "2021-07-27"

console_start_date = "2021-07-14"
console_end_date = "2021-07-27"

pre_post_son_user_pc = pre_load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='user_master').where(col("lastlogindate") >= pc_start_date)
pre_post_son_gcoin_pc = pre_load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use')
print('== PC DONE ==')

pre_post_son_user_console = pre_load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='user_master').where(col("lastlogindate") >= console_start_date)
pre_post_son_gcoin_console = pre_load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use')
print('== CONSOLE DONE ==')

== PC DONE ==
== CONSOLE DONE ==


PC

In [63]:
a = pre_post_son_user_pc.groupBy("date").agg(countDistinct("accountid").alias("au"))
b = pre_post_son_gcoin_pc.groupBy("date").agg(countDistinct("account_id").alias("pu"), sum("price").alias("revenue"), count("*").alias("cnt"))
c = a.join(b, "date", "left").withColumn("pur", col("pu")/col("au")).withColumn("arpu", col("revenue")/col("au")).withColumn("arppu", col("revenue")/col("pu"))

In [64]:
c.orderBy("date").toPandas()

,date,au,pu,revenue,cnt,pur,arpu,arppu
0,2021-07-14,1578972,71802,255608410,269322,0.045474,161.882801,3559.906549
1,2021-07-15,2233047,56150,158741780,180083,0.025145,71.087523,2827.102048
2,2021-07-16,2751827,53013,122582510,150661,0.019265,44.545864,2312.310377
3,2021-07-17,3242143,54029,115237120,145867,0.016665,35.543503,2132.875308
4,2021-07-18,3628332,49554,99372210,128898,0.013658,27.387849,2005.331759
5,2021-07-19,3888285,40001,79363420,103583,0.010288,20.410906,1984.035899
6,2021-07-20,4110673,35191,71202800,91985,0.008561,17.321446,2023.324145
7,2021-07-21,4311689,58048,124816780,134241,0.013463,28.948465,2150.233944
8,2021-07-22,4494625,40473,82227970,97505,0.009005,18.294734,2031.674697
9,2021-07-23,4674147,37852,80819140,95216,0.008098,17.290671,2135.135264


CONSOLE

In [65]:
a = pre_post_son_user_console.groupBy("date").agg(countDistinct("accountid").alias("au"))
b = pre_post_son_gcoin_console.groupBy("date").agg(countDistinct("account_id").alias("pu"), sum("price").alias("revenue"), count("*").alias("cnt"))
c = a.join(b, "date", "left").withColumn("pur", col("pu")/col("au")).withColumn("arpu", col("revenue")/col("au")).withColumn("arppu", col("revenue")/col("pu"))

In [66]:
c.orderBy("date").toPandas()

,date,au,pu,revenue,cnt,pur,arpu,arppu
0,2021-07-14,163220,2050,2616870,4500,0.012560,16.032778,1276.521951
1,2021-07-15,232259,13973,24144620,27923,0.060161,103.955584,1727.948186
2,2021-07-16,297850,11251,16555970,22008,0.037774,55.584925,1471.510977
3,2021-07-17,357729,8292,11422580,16603,0.023180,31.930819,1377.542209
4,2021-07-18,410110,6989,9186080,13997,0.017042,22.399064,1314.362570
5,2021-07-19,449050,4869,5863680,9460,0.010843,13.057967,1204.288355
6,2021-07-20,484825,4239,5048930,8666,0.008743,10.413923,1191.066289
7,2021-07-21,516782,4567,5965330,9111,0.008837,11.543223,1306.181301
8,2021-07-22,547892,4380,6046130,9362,0.007994,11.035259,1380.394977
9,2021-07-23,579900,4165,5617720,8956,0.007182,9.687394,1348.792317


## BlackPink

In [101]:
item_meta_sales.where(col("event_name") == "202108_blackpink").select("product_id").distinct().toPandas()["product_id"].tolist()

['itemdesc.13000316',
 'itemdesc.13000315',
 'itemdesc.13000318',
 'itemdesc.13000317',
 'itemdesc.13000314']

In [43]:
# bp_daily = all_economy_gcoin.where(col("product_id").isin(
#     item_meta_sales.where(col("event_name") == "202108_blackpink").select("product_id").distinct().toPandas()["product_id"].tolist()
# )).toPandas()
# bp_daily.to_csv(DATA_PATH + 'bp_daily.csv', index=False)
bp_daily = pd.read_csv(DATA_PATH + 'bp_daily.csv')

In [44]:
bp_daily.shape

(226316, 17)

In [71]:
pc_start_date = "2021-08-04"
pc_end_date = "2021-10-22"

console_start_date = "2021-08-12"
console_end_date = "2021-10-22"

event_name = "202108_blackpink"
bp_user_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='user_master').where(col("lastlogindate") >= pc_start_date)
bp_gcoin_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use').where(col("event_name") == event_name)
print('== PC DONE ==')

bp_user_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='user_master').where(col("lastlogindate") >= console_start_date)
bp_gcoin_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use').where(col("event_name") == event_name)
print('== CONSOLE DONE ==')

== PC DONE ==
== CONSOLE DONE ==


In [72]:
bp_period_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use')
bp_period_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use')

In [73]:
bp_pc_a = bp_user_pc.groupBy('platform').agg(countDistinct('accountid').alias('au')).toPandas()
bp_pc_b = bp_gcoin_pc.groupBy("platform").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()
bp_pc = pd.merge(bp_pc_a, bp_pc_b, on='platform', how="left").fillna(0)
bp_pc['pur'] = bp_pc.pu / bp_pc.au
bp_pc['arpu'] = bp_pc.revenue / bp_pc.au
bp_pc['arppu'] = bp_pc.revenue / bp_pc.pu

In [75]:
bp_pc.to_csv(DATA_PATH + "bp_pc.csv", index=False)

In [76]:
bp_pc

,platform,au,cnt,pu,revenue,pur,arpu,arppu
0,KAKAO,1087073,33943,30497,148295540,0.028054,136.417278,4862.627144
1,STEAM,10668806,252858,222662,1239964120,0.020870,116.223326,5568.817849
2,MAILRU,9700,16,14,51600,0.001443,5.319588,3685.714286


In [77]:
bp_console_a = bp_user_console.groupBy('platform').agg(countDistinct('accountid').alias('au')).toPandas()
bp_console_b = bp_gcoin_console.groupBy("platform").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()
bp_console = pd.merge(bp_console_a, bp_console_b, on='platform', how="left").fillna(0)
bp_console['pur'] = bp_console.pu / bp_console.au
bp_console['arpu'] = bp_console.revenue / bp_console.au
bp_console['arppu'] = bp_console.revenue / bp_console.pu

In [78]:
bp_console.to_csv(DATA_PATH + "bp_console.csv", index=False)

In [79]:
bp_console

,platform,au,cnt,pu,revenue,pur,arpu,arppu
0,XBOX,1076482,6859,5603,27398420,0.005205,25.451814,4889.955381
1,STADIA,12387,29,24,94860,0.001938,7.658029,3952.500000
2,PSN,531592,3358,2793,14112760,0.005254,26.548105,5052.903688


### 상품별

In [80]:
bp_product_gcoin_pc = bp_gcoin_pc.groupBy("product_name", "product_id").agg(countDistinct("transaction_id").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).withColumn("ARPPU", col("revenue")/col("pu")).orderBy("cnt", ascending=False).toPandas()
bp_product_gcoin_pc_platform = bp_gcoin_pc.groupBy("platform", "product_name", "product_id").agg(countDistinct("transaction_id").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).withColumn("ARPPU", col("revenue")/col("pu")).orderBy("platform", "cnt", ascending=False).toPandas()

In [81]:
bp_product_gcoin_console = bp_gcoin_console.groupBy("product_name", "product_id").agg(countDistinct("transaction_id").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).withColumn("ARPPU", col("revenue")/col("pu")).orderBy("cnt", ascending=False).toPandas()
bp_product_gcoin_console_platform = bp_gcoin_console.groupBy("platform", "product_name", "product_id").agg(countDistinct("transaction_id").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).withColumn("ARPPU", col("revenue")/col("pu")).orderBy("platform", "cnt", ascending=False).toPandas()

PC

In [82]:
bp_product_gcoin_pc

,product_name,product_id,cnt,pu,revenue,ARPPU
0,BLACKPINK MEGA BUNDLE,itemdesc.13000318,83000,83000,821700000,9900.0
1,JENNIE'S BLACKPINK BUNDLE,itemdesc.13000317,59618,59618,165738040,2780.0
2,ROSÉ'S BLACKPINK BUNDLE,itemdesc.13000314,58770,58770,163380600,2780.0
3,JISOO'S BLACKPINK BUNDLE,itemdesc.13000315,49207,49207,136795460,2780.0
4,LISA'S BLACKPINK BUNDLE,itemdesc.13000316,36222,36222,100697160,2780.0


In [83]:
bp_product_gcoin_pc_platform

,platform,product_name,product_id,cnt,pu,revenue,ARPPU
0,STEAM,BLACKPINK MEGA BUNDLE,itemdesc.13000318,75424,75424,746697600,9900.0
1,STEAM,JENNIE'S BLACKPINK BUNDLE,itemdesc.13000317,51008,51008,141802240,2780.0
2,STEAM,ROSÉ'S BLACKPINK BUNDLE,itemdesc.13000314,49772,49772,138366160,2780.0
3,STEAM,JISOO'S BLACKPINK BUNDLE,itemdesc.13000315,42887,42887,119225860,2780.0
4,STEAM,LISA'S BLACKPINK BUNDLE,itemdesc.13000316,33767,33767,93872260,2780.0
5,MAILRU,JENNIE'S BLACKPINK BUNDLE,itemdesc.13000317,8,8,22240,2780.0
6,MAILRU,JISOO'S BLACKPINK BUNDLE,itemdesc.13000315,5,5,13900,2780.0
7,MAILRU,ROSÉ'S BLACKPINK BUNDLE,itemdesc.13000314,2,2,5560,2780.0
8,MAILRU,BLACKPINK MEGA BUNDLE,itemdesc.13000318,1,1,9900,9900.0
9,KAKAO,ROSÉ'S BLACKPINK BUNDLE,itemdesc.13000314,8996,8996,25008880,2780.0


CONSOLE

In [84]:
bp_product_gcoin_console

,product_name,product_id,cnt,pu,revenue,ARPPU
0,JISOO'S BLACKPINK BUNDLE,itemdesc.13000315,2793,2793,7764540,2780.0
1,ROSÉ'S BLACKPINK BUNDLE,itemdesc.13000314,2737,2737,7608860,2780.0
2,JENNIE'S BLACKPINK BUNDLE,itemdesc.13000317,2160,2160,6004800,2780.0
3,BLACKPINK MEGA BUNDLE,itemdesc.13000318,1843,1843,18245700,9900.0
4,LISA'S BLACKPINK BUNDLE,itemdesc.13000316,713,713,1982140,2780.0


In [85]:
bp_product_gcoin_console_platform

,platform,product_name,product_id,cnt,pu,revenue,ARPPU
0,XBOX,JISOO'S BLACKPINK BUNDLE,itemdesc.13000315,1907,1907,5301460,2780.0
1,XBOX,ROSÉ'S BLACKPINK BUNDLE,itemdesc.13000314,1859,1859,5168020,2780.0
2,XBOX,JENNIE'S BLACKPINK BUNDLE,itemdesc.13000317,1456,1456,4047680,2780.0
3,XBOX,BLACKPINK MEGA BUNDLE,itemdesc.13000318,1170,1170,11583000,9900.0
4,XBOX,LISA'S BLACKPINK BUNDLE,itemdesc.13000316,467,467,1298260,2780.0
5,STADIA,JISOO'S BLACKPINK BUNDLE,itemdesc.13000315,10,10,27800,2780.0
6,STADIA,ROSÉ'S BLACKPINK BUNDLE,itemdesc.13000314,8,8,22240,2780.0
7,STADIA,JENNIE'S BLACKPINK BUNDLE,itemdesc.13000317,5,5,13900,2780.0
8,STADIA,LISA'S BLACKPINK BUNDLE,itemdesc.13000316,4,4,11120,2780.0
9,STADIA,BLACKPINK MEGA BUNDLE,itemdesc.13000318,2,2,19800,9900.0


## 해당 기간내 다른 상품들과 비교

In [86]:
bp_period_product_rank_pc = bp_period_pc.groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas()

In [87]:
bp_period_product_rank_pc.to_csv(DATA_PATH + "bp_period_product_rank_pc.csv", index=False)

In [88]:
bp_period_product_rank_console = bp_period_console.groupBy("product_name", "product_id").agg(avg("price").alias("price"), 
                                                                            count("*").alias("sell_cnt"),
                                                                            countDistinct("account_id").alias("PU"),
                                                                            sum("price").alias("revenue")
                                                                           ).withColumn("ARPPU", col("revenue")/col("PU")).orderBy("sell_cnt", ascending=False).toPandas()

In [89]:
bp_period_product_rank_console.to_csv(DATA_PATH + "bp_period_product_rank_console.csv", index=False)

### 콜라보 전후 1주일

In [90]:
pc_start_date = "2021-07-28"
pc_end_date = "2021-08-10"

console_start_date = "2021-08-05"
console_end_date = "2021-08-18"

pre_post_bp_user_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='user_master').where(col("lastlogindate") >= pc_start_date)
pre_post_bp_gcoin_pc = load_data_mart(start_date=pc_start_date, end_date=pc_end_date, device='pc', table_name='gcoin_use')
print('== PC DONE ==')

pre_post_bp_user_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='user_master').where(col("lastlogindate") >= console_start_date)
pre_post_bp_gcoin_console = load_data_mart(start_date=console_start_date, end_date=console_end_date, device='console', table_name='gcoin_use')
print('== CONSOLE DONE ==')

== PC DONE ==
== CONSOLE DONE ==


PC

In [91]:
a = pre_post_bp_user_pc.groupBy("date").agg(countDistinct("accountid").alias("au"))
b = pre_post_bp_gcoin_pc.groupBy("date").agg(countDistinct("account_id").alias("pu"), sum("price").alias("revenue"), count("*").alias("cnt"))
c = a.join(b, "date", "left").withColumn("pur", col("pu")/col("au")).withColumn("arpu", col("revenue")/col("au")).withColumn("arppu", col("revenue")/col("pu"))

In [93]:
c.orderBy("date").toPandas()

,date,au,pu,revenue,cnt,pur,arpu,arppu
0,2021-07-28,1581360,154813,208921670,323543,0.097899,132.115186,1349.509860
1,2021-07-29,2199607,75863,106063290,156699,0.034489,48.219200,1398.089846
2,2021-07-30,2667545,59157,86665850,125166,0.022177,32.488993,1465.014284
3,2021-07-31,3113196,57430,89459800,125770,0.018447,28.735679,1557.718962
4,2021-08-01,3479033,51395,81086090,112974,0.014773,23.307077,1577.703862
5,2021-08-02,3732852,39158,62025270,86519,0.010490,16.616054,1583.974411
6,2021-08-03,3953418,34987,53068310,75847,0.008850,13.423400,1516.800812
7,2021-08-04,4120117,73488,368935340,160724,0.017836,89.544870,5020.348084
8,2021-08-05,4304933,65742,232530800,140112,0.015271,54.014964,3537.020474
9,2021-08-06,4488953,53868,162773380,115454,0.012000,36.260879,3021.708250


CONSOLE

In [94]:
a = pre_post_bp_user_console.groupBy("date").agg(countDistinct("accountid").alias("au"))
b = pre_post_bp_gcoin_console.groupBy("date").agg(countDistinct("account_id").alias("pu"), sum("price").alias("revenue"), count("*").alias("cnt"))
c = a.join(b, "date", "left").withColumn("pur", col("pu")/col("au")).withColumn("arpu", col("revenue")/col("au")).withColumn("arppu", col("revenue")/col("pu"))

In [95]:
c.orderBy("date").toPandas()

,date,au,pu,revenue,cnt,pur,arpu,arppu
0,2021-08-05,162666,3910,4512240,7790,0.024037,27.739294,1154.025575
1,2021-08-06,232570,3805,4826500,7836,0.016361,20.752892,1268.462549
2,2021-08-07,289293,3469,4199000,7025,0.011991,14.514696,1210.435284
3,2021-08-08,348912,4091,4607620,8055,0.011725,13.205679,1126.282083
4,2021-08-09,391935,3089,3460730,6236,0.007881,8.829857,1120.339916
5,2021-08-10,430114,2657,3009050,5309,0.006177,6.995936,1132.499059
6,2021-08-11,463899,2518,2840170,5161,0.005428,6.122389,1127.946783
7,2021-08-12,487648,4217,13928340,12104,0.008648,28.562283,3302.902537
8,2021-08-13,515239,4138,10573960,10459,0.008031,20.522437,2555.331078
9,2021-08-14,546602,4176,9039530,10256,0.007640,16.537682,2164.638410
